In [5]:
import subprocess as sp
import os
from glob import glob
import shutil

In [6]:
# droidbot -keep_env -grant_perm -ignore_ad -count 10800 -a "Firefox Fast & Private Browser_125.3.0_APKPure.apk" -o output_dir_apk_name


def run_droidbot(apk_path: str, output_dir_path: str):
    assert os.path.exists(apk_path), "APK file not found"
    ret = sp.run(
        [
            "droidbot",
            "-keep_env",
            "-grant_perm",
            "-ignore_ad",
            "-count",
            "10800",
            "-a",
            apk_path,
            "-o",
            output_dir_path,
        ],
        capture_output=True,
    )
    if ret.returncode != 0:
        print(ret.stderr.decode("utf-8"))
        print("\n")
        print(ret.stdout.decode("utf-8"))
        return False
    return True

In [7]:
category = "tools"

input_root_dir = "apks"
output_root_dir = "utg"

input_dir = os.path.join(input_root_dir, category)
output_category_dir = os.path.join(output_root_dir, category)
os.makedirs(output_category_dir, exist_ok=True)

processing_dir = os.path.join(input_dir, "processing")
os.makedirs(processing_dir, exist_ok=True)
finished_dir = os.path.join(input_dir, "finished")
os.makedirs(finished_dir, exist_ok=True)

In [ ]:
def contains_apk_files(directory: str):
    return len(glob(os.path.join(directory, "*.apk"))) > 0

In [ ]:
def process_one_apk(input_dir, processing_dir, finished_dir, output_category_dir):
    apk_fps = glob(os.path.join(input_dir, "*.apk"))

    if len(apk_fps) > 0:
        apk_fp = apk_fps[0]
        apk_fn = os.path.basename(apk_fp)
        processing_fp = os.path.join(processing_dir, apk_fn)
        if os.path.exists(processing_fp):
            print(f"Already processing {apk_fp}")
            return
        finished_fp = os.path.join(finished_dir, apk_fn)
        if os.path.exists(finished_fp):
            print(f"Already finished {apk_fp}")
            return
        # begin processing
        shutil.move(apk_fp, processing_fp)
        ok = run_droidbot(processing_fp, os.path.join(output_category_dir, apk_fn))
        if ok:
            shutil.move(processing_fp, finished_fp)
        else:
            shutil.move(processing_fp, apk_fp)

In [ ]:
def main():
    cnt = 0
    while contains_apk_files(input_dir):
        process_one_apk(input_dir, processing_dir, finished_dir, output_category_dir)
        cnt += 1
        if cnt % 10 == 0:
            print(f"Processed {cnt} apks")
    print(f"Processed {cnt} apks")